In [1]:
##############################################################################
#The app works around 4 major steps
#    (1) Configure the user input
#    (2) Confirm the user input + submit the job
#    (3) Monitor the process during the run
#    (4) Display the final results
#
#These steps are linked via the following 
#    (1) --> (2): user_inputs, a dictionary of all the user inputs
#    (1) --> (2): mpuc3_code, a code for marketplace usercase 3
#    (2) --> (3): process, the process (MPusercase3 CalcJob) submitted to AiiDA 
#    (3) --> (4): output, the ArrayData output from the CalcJob
#
#This is an early prototype that is 'stapled' from the pizza example in aiidalab_widgets_base
#as well as QE example from aiidalab_qe
#
#Specific notes/issues on the steps:
#    Step 1:
#        * Perhaps there should be a step 0 that allows the user to choose a model,
#          then the widget can be dynamically updated
#        * The code setup is rather ugly, and creating a custom code crashes
#        * The user should be alerted that if they try to hit submit without 
#          choosing a code, that they need to choose a code first
#    Step 2:
#    Step 3:
#        * This is taken almost directly from QE, and is the part I understand the least
#        * Clicking on most outputs causes a crash
#        *** I can't seem to set the 'output' propery correctly
#    Step 4:
#        *** Because I can't set the output property in step3 correctly, this step displays nothing
#        * The current display is very ugly and taken directly from Step2, and probably be changed to
#         something nicer
##############################################################################

In [2]:
import logging

from uc3.logger import OutputWidgetHandler

logger = logging.getLogger("aiidalab_mp_uc3")
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

# Remove/Add comment of the next line to show/hide logs.
# handler.show_logs()

handler.clear_logs()
logger.info('Starting program')

In [3]:
import ipywidgets as ipw

from aiidalab_widgets_base import WizardAppWidget

from uc3.steps import (
    ComputerCodeSetupStep,
    ConfigureUserInputStep,
    ConfirmUserInputStep,
    MonitorProcessStep,
    DisplayAp2FinalOutput,
)

computer_code_setup_step = ComputerCodeSetupStep(auto_advance=True)
configure_userinput_step = ConfigureUserInputStep(auto_advance=True, description_label_default=[  
  ['dmf', 'Diluent mass fraction', 0.9],
  ['gfr', 'Gas Flow Rate (mL/min)', 100],
  ['ch4mf', 'CH4 Mole Fraction', 0.02],
  ['h2omf', 'H20 Mole Fraction', 0],
  ['surfa', 'Specific Surface Area (m^2/g)', 100],
  ['amf', 'Active Mass Fraction', 0.01],
  ['macropor', 'Catalyst Macroporosity', 0.1],
  ['macrotor', 'Catalyst Tortuosity', 2],
  ['k1', 'k1 (m/s)', 20000],
  ['ea1', 'Ea1 (J/mol)', 10000],
  ['k2', 'k2 (m^7 / mol^2 / s)', 60000000],
  ['ea2', 'Ea2 (J/mol)', 150000],
  ['koeq', 'K0eq (m^6/mol^2)', 2000],
  ['dhh20', 'dH,H20 (J/mol)', -5000]
])


confirm_userinput_step = ConfirmUserInputStep(auto_advance=True)
monitorprocess_step = MonitorProcessStep(auto_advance=True)
displayfinaloutput_step = DisplayAp2FinalOutput(auto_advance=False) # DisplayFinalOutput(auto_advance=False)

ipw.dlink(
    (configure_userinput_step, "user_inputs"),
    (confirm_userinput_step, "user_inputs"),
)
ipw.dlink(
    (computer_code_setup_step, "mpuc3_code"),
    (confirm_userinput_step, "mpuc3_code"),
)
ipw.dlink(
    (confirm_userinput_step, "process"),
    (monitorprocess_step, "process"),
    transform=lambda x: x.uuid if x is not None else None
)
ipw.dlink(
    (monitorprocess_step, "output"),
    (displayfinaloutput_step, "output"),
)

# Setup the app by adding the various steps in order.
WizardAppWidget(
    steps=[
        ("Setup Computer & Code", computer_code_setup_step),
        ("Configure User Input", configure_userinput_step),
        ("Confirm User Input", confirm_userinput_step),
        ("Monitor Process", monitorprocess_step),
        ("Display Results", displayfinaloutput_step),
    ]
)

NameError: name 'DisplayAp2FinalOutput' is not defined